### Azure Ai Agents tracing with Azure Monitor, OpenTelemetry, App Insights.

In [1]:
"""
DESCRIPTION:
    This sample demonstrates how to use basic agent operations from
    the Azure Agents service using a synchronous client with Azure Monitor tracing.
    View the results in the "Tracing" tab in your Azure AI Foundry project page.

USAGE:
    python sample_agents_basics_with_azure_monitor_tracing.py

    Before running the sample:

    pip install azure-ai-projects azure-ai-agents azure-identity azure-monitor-opentelemetry

    Set these environment variables with your own values:
    1) PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
                          page of your Azure AI Foundry portal.
    2) MODEL_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED - Optional. Set to `true` to trace the content of chat
       messages, which may contain personal data. False by default.
    4) APPLICATIONINSIGHTS_CONNECTION_STRING - Set to the connection string of your Application Insights resource.
       This is used to send telemetry data to Azure Monitor. You can also get the connection string programmatically
       from AIProjectClient using the `telemetry.get_connection_string` method. A code sample showing how to do this
       can be found in the `sample_telemetry.py` file in the azure-ai-projects telemetry samples.
"""

import os, time
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient

import dotenv
# Load environment variables from .env file
dotenv.load_dotenv()

True

In [2]:
# [START enable_tracing]
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

# Enable Azure Monitor tracing
application_insights_connection_string = os.environ["APPLICATIONINSIGHTS_CONNECTION_STRING"]
configure_azure_monitor(connection_string=application_insights_connection_string)

# add the date & time to the scenario name
from datetime import datetime
date_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Scenario: AI-Agent-Trace-Monitoring-{date_str}")
scenario = os.path.basename(f"AI-Agent-Trace-Monitoring-{date_str}")
tracer = trace.get_tracer("monitoring-sample", "1.0.0")

Scenario: AI-Agent-Trace-Monitoring-2025-06-14 10:18:49


In [3]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [4]:
with tracer.start_as_current_span(scenario):
    with project_client:

        # [END enable_tracing]
        agent = project_client.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"], name="my-agent-with-tracing", instructions="You are helpful agent"
        )
        print(f"Created agent, agent ID: {agent.id}")

        thread = project_client.threads.create()
        print(f"Created thread, thread ID: {thread.id}")

        message = project_client.messages.create(
            thread_id=thread.id, role="user", content="Hello, what is the easiest way to learn Python?"
        )
        print(f"Created message, message ID: {message.id}")

        run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
        print(f"Run completed with status: {run.status}")

        project_client.delete_agent(agent.id)
        print("Deleted agent")

        messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        for msg in messages:
            if msg.text_messages:
                last_text = msg.text_messages[-1]
                print(f"{msg.role}: {last_text.text.value}")

Created agent, agent ID: asst_mQqGtbVVRQ62GIi9BouThLwE
Created thread, thread ID: thread_1aUJKwSSdmVfuODMhXdDl3ar
Created message, message ID: msg_kw4erWrJt5rGR2rDwCPKXTTl
Run completed with status: RunStatus.COMPLETED
Deleted agent
MessageRole.USER: Hello, what is the easiest way to learn Python?
MessageRole.AGENT: Hello! Learning Python can be both fun and accessible, especially with the right approach. Here’s an easy way to get started:

### 1. **Set Up Your Environment**
- **Install Python:** Download from the [official Python website](https://www.python.org/downloads/).
- **IDE/Text Editor:** Use beginner-friendly options like [Thonny](https://thonny.org/), [VS Code](https://code.visualstudio.com/), or even the default IDLE.

### 2. **Learn Basic Concepts**
- **Variables and Data Types:** Learn about numbers, strings, lists, dictionaries, etc.
- **Control Flow:** Understand if-else statements, for and while loops.
- **Functions:** Learn how to define and use functions.

### 3. **U

In [5]:


# add the date & time to the scenario name
from datetime import datetime
date_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Scenario: AI-Agent-Trace-Monitoring-{date_str}")
scenario = os.path.basename(f"AI-Agent-Trace-Monitoring-{date_str}")
tracer = trace.get_tracer("monitoring-sample", "1.0.0")

Scenario: AI-Agent-Trace-Monitoring-2025-06-14 10:19:02


In [6]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [7]:
with tracer.start_as_current_span(scenario):
    with project_client:

        # [END enable_tracing]
        agent = project_client.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"], name="my-agent-with-tracing", instructions="You are helpful agent"
        )
        print(f"Created agent, agent ID: {agent.id}")

        thread = project_client.threads.create()
        print(f"Created thread, thread ID: {thread.id}")

        message = project_client.messages.create(
            thread_id=thread.id, role="user", content="what is the laplace transform used for"
        )
        print(f"Created message, message ID: {message.id}")

        run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
        print(f"Run completed with status: {run.status}")

        project_client.delete_agent(agent.id)
        print("Deleted agent")

        messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        for msg in messages:
            if msg.text_messages:
                last_text = msg.text_messages[-1]
                print(f"{msg.role}: {last_text.text.value}")

Created agent, agent ID: asst_WuolavXYiqo6LoIvW5S0tRHl
Created thread, thread ID: thread_oYQUDoxu0Q8dJNd550XnJFsQ
Created message, message ID: msg_2QIsgjXFrKPUBTJaeMtmssfY
Run completed with status: RunStatus.COMPLETED
Deleted agent
MessageRole.USER: what is the laplace transform used for
MessageRole.AGENT: The **Laplace transform** is a powerful mathematical tool used primarily to simplify the analysis of linear systems, especially in engineering, physics, and mathematics. Here’s what it’s used for:

---

### **1. Solving Differential Equations**
- The Laplace transform converts complex **differential equations** (which are difficult to solve directly) into **algebraic equations** (which are simpler to handle).
- Once solved in the Laplace (frequency) domain, the inverse Laplace transform converts the solution back to the original time domain.

---

### **2. Control Systems Engineering**
- Used to analyze and design **control systems** (like those in robotics, industrial automation, a